<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Subsetting-posts-by-contraceptive-first." data-toc-modified-id="Subsetting-posts-by-contraceptive-first.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Subsetting posts by contraceptive first.</a></span></li><li><span><a href="#Let's-compare-topic-modeling-with-an-LDA-vs-an-NMF-with-the-posts-from-each-contraceptive-type." data-toc-modified-id="Let's-compare-topic-modeling-with-an-LDA-vs-an-NMF-with-the-posts-from-each-contraceptive-type.-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Let's compare topic modeling with an LDA vs an NMF with the posts from each contraceptive type.</a></span></li></ul></div>

In [1]:
import pandas as pd
import os
import numpy as np
import functools

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from functools import reduce

In [38]:
path = "/Users/krittikakrishnan/Desktop/Insight Project"
os.chdir(path)
df = pd.read_csv("All Uncleaned Reddit Posts.csv", delimiter = ",")

In [39]:
#Standardizing the original selftext
def standardize_text(new, text_field):
    new[text_field] = new[text_field].str.replace(r"http\S+", "")
    new[text_field] = new[text_field].str.replace(r"http", "")
    new[text_field] = new[text_field].str.replace(r"@\S+", "")
    new[text_field] = new[text_field].str.replace(r"\n", "")
    new[text_field] = new[text_field].str.replace(r"[^A-Za-z0-9 (),.!?@\'\`\"\_\n]", "")
    new[text_field] = new[text_field].str.replace(r"@", "at")
    new[text_field] = new[text_field].str.lower()
    return new

# new = standardize_text(uncleaned, "title")
posts = standardize_text(df, "selftext")

In [40]:
posts

,Unnamed: 0,selftext,id
0,0,i went off birth control around the beginning ...,8s5kam
1,1,update got my period today it was over a week ...,8s5s7t
2,2,i got off the pill on may 27 after two years. ...,8s67mh
3,3,it's a rather broad question and applies to an...,8s6n5g
4,4,my bf and i recently had a little bit of a sca...,8s6r3k
5,5,okay so today i had sex in the car (really bad...,8s6sb4
6,6,so i've had my mirena in for just over two mon...,8s6wg7
7,7,no text available,8s6wv1
8,8,i started taking yaz last week. today i had my...,8s7d94
9,9,"currently, im unable to take the combined pill...",8s7dy6


In [41]:
reddit = posts.drop(["Unnamed: 0"], axis = 1)

In [51]:
reddit.head(10)

,selftext,id
0,i went off birth control around the beginning ...,8s5kam
1,update got my period today it was over a week ...,8s5s7t
2,i got off the pill on may 27 after two years. ...,8s67mh
3,it's a rather broad question and applies to an...,8s6n5g
4,my bf and i recently had a little bit of a sca...,8s6r3k
5,okay so today i had sex in the car (really bad...,8s6sb4
6,so i've had my mirena in for just over two mon...,8s6wg7
7,no text available,8s6wv1
8,i started taking yaz last week. today i had my...,8s7d94
9,"currently, im unable to take the combined pill...",8s7dy6


In [ ]:
no_features = 100

In [ ]:
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(documents[doc_index])

In [ ]:
###Running the TF-IDF Count Vectorizer in prep for Latent Dirichlet Allocation (LDA) for topic modeling

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(sentences["SENTENCE"])
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
no_topics = 5

lda_model = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [ ]:
no_top_words = 5
no_top_documents = 2
display_topics(lda_H, lda_W, tf_feature_names, sentences["SENTENCE"], no_top_words, no_top_documents)

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 15
display_topics(lda, tf_feature_names, no_top_words)

This is really cool! I would love to display the topics associated with the side effect & the contraceptive. Going to try doing that below!

# Subsetting posts by contraceptive first.

In [34]:
##Trying to automate dataframe extraction with a function and for loop. NOT CURRENTLY WORKING.
#FIX LATER.
##There has to be a quick way to do this. Spend time on this later. 
types = ["iud", "pill", "patch", "ring", "tubal ligation", "condom", "implant", "depo"]
def counting_types(dataFrame, text_field, types):
    d = {}
    for i in range(dataFrame.shape[0]):
        entry = dataFrame[text_field][i]
        for j in range(len(types)):
#             print(entry)
            this_type = types[j]
            if this_type in entry:
#                 print(this_effect)
                d[this_type] = pd.DataFrame(dataFrame[text_field].str.contains(this_type))
#                 number[i,j] = 1
    return d

In [ ]:
type_count = counting_types(reddit, "selftext", types)

In [ ]:
IUD = reddit[reddit['selftext'].str.contains("iud", na = False)]
IUD = pd.DataFrame(IUD)

In [ ]:
filters = [("pill", False), ("iud", True), ("patch", False), ("condom", False), ("ring", False), 
           ("tubal ligation", False), ("depo", False), ("implant", False)]
new_IUD = reduce(lambda IUD, f: IUD[IUD["selftext"].str.contains(f[0]) == f[1]], filters, IUD)


In [ ]:
#reset index for subsetted dataframe
IUD_1 = new_IUD.reset_index(drop=True)

In [53]:
reddit.columns

Index(['selftext', 'id'], dtype='object')

In [ ]:
for i in range(dataFrame.shape[0]):
        entry = dataFrame[text_field][i]

In [80]:
types = ["iud", "pill", "patch", "ring", "tubal ligation", "condom", "implant", "depo"]

def counting_types2(dataFrame, item):
   
    df = dataFrame[dataFrame["selftext"].str.contains(item)]
     
    for word in types:
        if word != item:
            df = df[~df["selftext"].str.contains(word)]

    return df

In [81]:
d = {}
for entry in types:
    d[entry] = counting_types2(reddit, entry)

In [82]:
d["pill"]

,selftext,id
2,i got off the pill on may 27 after two years. ...,8s67mh
4,my bf and i recently had a little bit of a sca...,8s6r3k
14,hey huysi have been taking cerelle pill for 4 ...,8s8hy7
17,this is my first month on the combined pill. i...,8s8xoz
23,no im serious. ive been on the same bcp since ...,8samx5
29,i am on mylan which is an off brand of seasoni...,8sbwml
32,so ive been on lolo combination pill birth con...,8scl6f
35,i started a new pack 3 days ago. the first day...,8sd0m2
45,"my boobs grew when i went on the pill, if i st...",8senc1
47,i am on gianvi (28 day regimen). i have taken ...,8rwbfp


In [78]:
pill_df = counting_types2(reddit, "pill")
pill_df.head()

20922
6673
4975
4898
3343
3343
2841
2655
2597
20922


,selftext,id
2,i got off the pill on may 27 after two years. ...,8s67mh
4,my bf and i recently had a little bit of a sca...,8s6r3k
14,hey huysi have been taking cerelle pill for 4 ...,8s8hy7
17,this is my first month on the combined pill. i...,8s8xoz
23,no im serious. ive been on the same bcp since ...,8samx5


In [35]:
pills = reddit[reddit['selftext'].str.contains("pill", na = False)]
pills = pd.DataFrame(pills)
# pill.to_csv('pill_Data.csv')

filters_pill = [("pill", True), ("iud", False), ("patch", False), ("condom", False), ("ring", False), 
           ("tubal ligation", False), ("depo", False), ("implant", False)]
new_pill = reduce(lambda pills, f: pills[pills["selftext"].str.contains(f[0]) == f[1]], filters_pill, pills)

#reset index for subsetted dataframe
pills_1 = new_pill.reset_index(drop=True)

In [36]:
pills_1.head(10)

,selftext,id
0,I got off the pill on May 27 after two years. ...,8s67mh
1,My bf and I recently had a little bit of a sca...,8s6r3k
2,Hi guys! Looking for advice from anyway who ha...,8s87fp
3,Hey huysI have been taking Cerelle pill for 4 ...,8s8hy7
4,This is my first month on the combined pill. I...,8s8xoz
5,Im looking for advice. Gf just finished period...,8s9sz7
6,No Im serious. Ive been on the same BCP since ...,8samx5
7,I am on mylan which is an off brand of seasoni...,8sbwml
8,So Ive been on lolo combination pill birth con...,8scl6f
9,I started a new pack 3 days ago. The first day...,8sd0m2


In [ ]:
condom = reddit[reddit['selftext'].str.contains("condoms", na = False)]
condom = pd.DataFrame(condom)
# condom.to_csv('condom_Data.csv')

filters_condom = [("pill", False), ("iud", False), ("patch", False), ("condoms", True), ("ring", False), 
           ("tubal ligation", False), ("depo", False), ("implant", False)]
new_condom = reduce(lambda condom, f: condom[condom["selftext"].str.contains(f[0]) == f[1]], filters_condom, condom)


#reset index for subsetted dataframe
condom_1 = new_condom.reset_index(drop=True)

In [ ]:
patch = reddit[reddit['selftext'].str.contains("patch", na = False)]
patch = pd.DataFrame(patch)
# patch.to_csv('patch_Data.csv')

filters_patch = [("pill",False), ("iud", False), ("patch", True), ("condom", False), ("ring", False), 
           ("tubal ligation", False), ("depo", False), ("implant", False)]
new_patch = reduce(lambda patch, f: patch[patch["selftext"].str.contains(f[0]) == f[1]], filters_patch, patch)

#reset index for subsetted dataframe
patch_1 = new_patch.reset_index(drop=True)

In [ ]:
implant = reddit[reddit['selftext'].str.contains("implant", na = False)]
implant = pd.DataFrame(implant)
# implant.to_csv('implant_Data.csv')

filters_implant = [("pill",False), ("iud", False), ("patch", False), ("condom", False), ("ring", False), 
           ("tubal ligation", False), ("depo", False), ("implant", True)]
new_implant = reduce(lambda implant, f: implant[implant["selftext"].str.contains(f[0]) == f[1]], filters_implant, implant)

#reset index for subsetted dataframe
implant_1 = new_implant.reset_index(drop=True)

In [ ]:
ring = reddit[reddit['selftext'].str.contains("ring", na = False)]
ring = pd.DataFrame(ring)
# ring.to_csv('ring_Data.csv')

filters_ring = [("pill",False), ("iud", False), ("patch", False), ("condom", False), ("ring", True), 
           ("tubal ligation", False), ("depo", False), ("implant", False)]
new_ring = reduce(lambda ring, f: ring[ring["selftext"].str.contains(f[0]) == f[1]], filters_ring, ring)

#reset index for subsetted dataframe
ring_1 = new_ring.reset_index(drop=True)

In [ ]:
depo = reddit[reddit['selftext'].str.contains("depo", na = False)]
depo = pd.DataFrame(depo)
# ring.to_csv('ring_Data.csv')

filters_depo = [("pill",False), ("iud", False), ("patch", False), ("condom", False), ("ring", False), 
           ("tubal ligation", False), ("depo", True), ("implant", False)]
new_depo = reduce(lambda depo, f: depo[depo["selftext"].str.contains(f[0]) == f[1]], filters_depo, depo)

#reset index for subsetted dataframe
depo_1 = new_depo.reset_index(drop=True)

In [ ]:
tubes_tied = reddit[reddit['selftext'].str.contains("tube", na = False)]
tubes_tied = pd.DataFrame(tubes_tied)
# tubes_tied.to_csv('tubes_tied_Data.csv')

filters_tubes = [("pill",False), ("iud", False), ("patch", False), ("condom", False), ("ring", False), 
           ("tube", True), ("depo", False), ("implant", False)]
new_tubes = reduce(lambda tubes_tied, f: tubes_tied[tubes_tied["selftext"].str.contains(f[0]) == f[1]], filters_tubes, tubes_tied)


#reset index for subsetted dataframe
tubes_tied_1 = new_tubes.reset_index(drop=True)

# Let's compare topic modeling with an LDA vs an NMF with the posts from each contraceptive type.

In [ ]:
no_features = 100    # Restrict the number of terms in the bag of words matrix to 100
no_topics = 10       # Create 10 topics
no_top_words = 5     # Display 5 words per topic
no_top_documents = 3 # Display the top 3 documents

###Running the TF-IDF Count Vectorizer in prep for Latent Dirichlet Allocation (LDA) for topic modeling
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')

###Running the TF-IDF Vectorizer in prep for non-Negative Matrix Factorization (NMF) for topic modeling
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')

In [ ]:
no_top_words = 5     # Display 5 words per topic
no_top_documents = 3 # Display the top 3 documents

def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    topic_title_list = []
    topic_text_list = []
    for topic_idx, topic in enumerate(H):
#          print("Topic %d:" % (topic_idx))
        topic_title_list.append(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            topic_text_list.append(documents[doc_index])
            
    return topic_title_list, topic_text_list

In [ ]:
tf_iud = tf_vectorizer.fit_transform(IUD_1["selftext"])
tf_iud_feature_names = tf_vectorizer.get_feature_names()
lda_iud = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf_iud)
lda_iud_W = lda_iud.transform(tf_iud)
lda_iud_H = lda_iud.components_
IUD_title_list, IUD_text_list = display_topics(lda_iud_H, lda_iud_W, tf_iud_feature_names, IUD_1["selftext"], no_top_words, no_top_documents)

In [ ]:
IUD_title_list

In [ ]:
tfidf_iud = tfidf_vectorizer.fit_transform(IUD_1["selftext"])
tfidf_iud_feature_names = tfidf_vectorizer.get_feature_names()
nmf_iud = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf_iud)
nmf_iud_W = nmf_iud.transform(tfidf_iud)
nmf_iud_H = nmf_iud.components_
IUD_NMF_titles, IUD_NMF_text = display_topics(nmf_iud_H, nmf_iud_W, tfidf_iud_feature_names, IUD_1["selftext"], no_top_words, no_top_documents)

In [ ]:
IUD_NMF_titles

In [ ]:
nmf_iud_W[0]
lda_iud_sum = np.sum(lda_iud_W, axis = 0)
np.argmax(lda_iud_sum)

In [ ]:
lda_iud_W[0]

In [ ]:
lda_iud_sum = np.sum(lda_iud_W, axis = 0)

In [ ]:
np.argmax(lda_iud_sum)

In [ ]:
IUD_top_topics = IUD_title_list

In [ ]:
IUD_top_topics

In [ ]:
b = [0,1,2]
IUD_top_docs = [IUD_text_list[i] for i in b]

In [ ]:
IUD_top_docs

In [ ]:
IUD_titles = pd.DataFrame(IUD_top_topics).reset_index()
IUD_text = pd.DataFrame(IUD_top_docs).reset_index()

IUD_text.columns = ['index', 'IUD']
IUD_titles.columns = ['index', 'IUD']

In [ ]:
tf_pills = tf_vectorizer.fit_transform(pills_1["selftext"])
tf_pills_feature_names = tf_vectorizer.get_feature_names()
lda_pills = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf_pills)
lda_pills_W = lda_pills.transform(tf_pills)
lda_pills_H = lda_pills.components_
pills_title_list, pills_text_list = display_topics(lda_pills_H, lda_pills_W, tf_pills_feature_names, pills_1["selftext"], no_top_words, no_top_documents)

In [ ]:
lda_pills_sum = np.sum(lda_pills_W, axis = 0)

In [ ]:
lda_pills_sum

In [ ]:
np.argmax(lda_pills_sum)

In [ ]:
pills_top_topics = pills_title_list

In [ ]:
b = [27,28,29]
pills_top_docs = [pills_text_list[i] for i in b]

In [ ]:
pills_titles = pd.DataFrame(pills_top_topics).reset_index()
pills_text = pd.DataFrame(pills_top_docs).reset_index()

pills_text.columns = ['index', 'Pills']
pills_titles.columns = ['index', 'Pills']

In [ ]:
tf_ring = tf_vectorizer.fit_transform(ring_1["selftext"])
tf_ring_feature_names = tf_vectorizer.get_feature_names()
lda_ring = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf_ring)
lda_ring_W = lda_ring.transform(tf_ring)
lda_ring_H = lda_ring.components_
ring_title_list, ring_text_list = display_topics(lda_ring_H, lda_ring_W, tf_ring_feature_names, ring_1["selftext"], no_top_words, no_top_documents)

In [ ]:
lda_ring_sum = np.sum(lda_ring_W, axis = 0)

In [ ]:
lda_ring_sum

In [ ]:
np.argmax(lda_ring_sum)

In [ ]:
ring_top_topics = ring_title_list

In [ ]:
b = [6,7,8]
ring_top_docs = [ring_text_list[i] for i in b]

In [ ]:
ring_titles = pd.DataFrame(ring_top_topics).reset_index()
ring_text = pd.DataFrame(ring_top_docs).reset_index()

ring_text.columns = ['index', 'Vaginal ring']
ring_titles.columns = ['index', 'Vaginal ring']

In [ ]:
tf_depo = tf_vectorizer.fit_transform(depo_1["selftext"])
tf_depo_feature_names = tf_vectorizer.get_feature_names()
lda_depo = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf_depo)
lda_depo_W = lda_depo.transform(tf_depo)
lda_depo_H = lda_depo.components_
depo_title_list, depo_text_list = display_topics(lda_depo_H, lda_depo_W, tf_depo_feature_names, depo_1["selftext"], no_top_words, no_top_documents)

In [ ]:
lda_depo_sum = np.sum(lda_depo_W, axis = 0)

In [ ]:
lda_depo_sum

In [ ]:
np.argmax(lda_depo_sum)

In [ ]:
depo_top_topics = depo_title_list

In [ ]:
b = [0,1,2]
depo_top_docs = [depo_text_list[i] for i in b]

In [ ]:
depo_titles = pd.DataFrame(depo_top_topics).reset_index()
depo_text = pd.DataFrame(depo_top_docs).reset_index()

depo_text.columns = ['index', 'Depo-Provera']
depo_titles.columns = ['index', 'Depo-Provera']

In [ ]:
tf_implant = tf_vectorizer.fit_transform(implant_1["selftext"])
tf_implant_feature_names = tf_vectorizer.get_feature_names()
lda_implant = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf_implant)
lda_implant_W = lda_implant.transform(tf_implant)
lda_implant_H = lda_implant.components_
implant_title_list, implant_text_list = display_topics(lda_implant_H, lda_implant_W, tf_implant_feature_names, implant_1["selftext"], no_top_words, no_top_documents)


In [ ]:
lda_implant_sum = np.sum(lda_implant_W, axis = 0)

In [ ]:
lda_implant_sum

In [ ]:
np.argmax(lda_implant_sum)

In [ ]:
implant_top_topics = implant_title_list

In [ ]:
b = [15,16,17]
implant_top_docs = [implant_text_list[i] for i in b]

In [ ]:
implant_titles = pd.DataFrame(implant_top_topics).reset_index()
implant_text = pd.DataFrame(implant_top_docs).reset_index()

implant_text.columns = ['index', 'Implant']
implant_titles.columns = ['index', 'Implant']

In [ ]:
tf_patch = tf_vectorizer.fit_transform(patch_1["selftext"])
tf_patch_feature_names = tf_vectorizer.get_feature_names()
lda_patch = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf_patch)
lda_patch_W = lda_patch.transform(tf_patch)
lda_patch_H = lda_patch.components_
patch_title_list, patch_text_list = display_topics(lda_patch_H, lda_patch_W, tf_patch_feature_names, patch_1["selftext"], no_top_words, no_top_documents)

In [ ]:
lda_patch_sum = np.sum(lda_patch_W, axis = 0)

In [ ]:
lda_patch_sum

In [ ]:
np.argmax(lda_patch_sum)

In [ ]:
patch_top_topics = patch_title_list

In [ ]:
b = [0,1,2]
patch_top_docs = [patch_text_list[i] for i in b]

In [ ]:
patch_titles = pd.DataFrame(patch_top_topics).reset_index()
patch_text = pd.DataFrame(patch_top_docs).reset_index()

patch_text.columns = ['index', 'Patch']
patch_titles.columns = ['index', 'Patch']

In [ ]:
tf_condom = tf_vectorizer.fit_transform(condom_1["selftext"])
tf_condom_feature_names = tf_vectorizer.get_feature_names()
lda_condom = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf_condom)
lda_condom_W = lda_condom.transform(tf_condom)
lda_condom_H = lda_condom.components_
condom_title_list, condom_text_list = display_topics(lda_condom_H, lda_condom_W, tf_condom_feature_names, condom_1["selftext"], no_top_words, no_top_documents)


In [ ]:
lda_condom_sum = np.sum(lda_condom_W, axis = 0)

In [ ]:
lda_condom_sum

In [ ]:
np.argmax(lda_condom_sum)

In [ ]:
condom_top_topics = condom_title_list

In [ ]:
b = [24,25,26]
condom_top_docs = [condom_text_list[i] for i in b]

In [ ]:
condom_titles = pd.DataFrame(condom_top_topics).reset_index()
condom_text = pd.DataFrame(condom_top_docs).reset_index()

condom_text.columns = ['index', 'Condom']
condom_titles.columns = ['index', 'Condom']

In [ ]:
tf_tubes = tf_vectorizer.fit_transform(tubes_tied_1["selftext"])
tf_tubes_feature_names = tf_vectorizer.get_feature_names()
lda_tubes = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf_tubes)
lda_tubes_W = lda_tubes.transform(tf_tubes)
lda_tubes_H = lda_tubes.components_
tubes_title_list, tubes_text_list = display_topics(lda_tubes_H, lda_tubes_W, tf_tubes_feature_names, tubes_tied_1["selftext"], no_top_words, no_top_documents)

In [ ]:
lda_tubes_sum = np.sum(lda_tubes_W, axis = 0)

In [ ]:
lda_tubes_sum

In [ ]:
np.argmax(lda_tubes_sum)

In [ ]:
tubes_top_topics = tubes_title_list

In [ ]:
b = [6,7,8]
tubes_top_docs = [tubes_text_list[i] for i in b]

In [ ]:
tubes_titles = pd.DataFrame(tubes_top_topics).reset_index()
tubes_text = pd.DataFrame(tubes_top_docs).reset_index()

tubes_text.columns = ['index', 'Tubal ligation']
tubes_titles.columns = ['index', 'Tubal ligation']

In [ ]:
##Join all the dataframes
text_data_frames = [tubes_text, IUD_text, pills_text, patch_text,
              implant_text, ring_text, condom_text, depo_text]

text_merged = functools.reduce(lambda  left,right: pd.merge(left,right,on=['index'],
                                            how='outer'), text_data_frames)

In [ ]:
#Join all the dataframes
titles_data_frames = [tubes_titles, IUD_titles, pills_titles, patch_titles,
              implant_titles, ring_titles, condom_titles, depo_titles]

titles_merged = functools.reduce(lambda  left,right: pd.merge(left,right,on=['index'],
                                            how='outer'), titles_data_frames)

In [ ]:
text_merged.to_csv("All Contraceptive Text.csv")

In [ ]:
titles_merged.to_csv("All Contraceptive Topics.csv")